In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix

In [3]:
def process_data(csv_file):
    data = pd.read_csv(csv_file)
    labels = data.iloc[:, 0].values
    images = data.iloc[:, 1:].values.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
    return images, labels

images, labels = process_data('train.csv')

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2)


In [13]:
test_data=pd.read_csv('test.csv')
images_t = test_data.iloc[:, :].values.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

2023-12-05 22:50:51.415853: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 22:50:51.422426: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
model.compile(optimizer='sgd',  # Use SGD optimizer
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
420/420 [==============================] - 5s 11ms/step - loss: 1.2019 - accuracy: 0.6899 - val_loss: 0.5158 - val_accuracy: 0.8286
Epoch 2/5
420/420 [==============================] - 4s 10ms/step - loss: 0.3363 - accuracy: 0.8985 - val_loss: 0.2716 - val_accuracy: 0.9214
Epoch 3/5
420/420 [==============================] - 4s 10ms/step - loss: 0.2414 - accuracy: 0.9271 - val_loss: 0.2108 - val_accuracy: 0.9381
Epoch 4/5
420/420 [==============================] - 4s 10ms/step - loss: 0.1916 - accuracy: 0.9415 - val_loss: 0.1827 - val_accuracy: 0.9433
Epoch 5/5
420/420 [==============================] - 4s 10ms/step - loss: 0.1594 - accuracy: 0.9513 - val_loss: 0.1508 - val_accuracy: 0.9545


In [7]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

263/263 [==============================] - 1s 3ms/step - loss: 0.1566 - accuracy: 0.9543
Test Loss: 0.15655754506587982, Test Accuracy: 0.954285740852356


In [11]:
predictions = np.argmax(model.predict(test_images), axis=1)
conf_matrix = confusion_matrix(test_labels, predictions)
print("Confusion Matrix:")
print(conf_matrix)

263/263 [==============================] - 1s 4ms/step
Confusion Matrix:
[[822   0   1   0   0   3   8   0   8   3]
 [  0 894   2   3   1   0   0   0   4   0]
 [  5   7 776   6   5   2   6   5  17   1]
 [  1   0  12 860   0   9   1   2  18  12]
 [  0   1   1   0 725   0  10   1   2  27]
 [  2   2   0  12   0 690   9   1   6   7]
 [  7   0   0   0   0   6 814   0   4   0]
 [  2   3  13   6   5   0   0 813   3  30]
 [  1   7   3   9   3   9   7   0 810  14]
 [  2   4   0   4   4   4   0   4   7 812]]


In [46]:
fins=np.argmax(model.predict(images_t), axis=1)

875/875 [==============================] - 2s 2ms/step


In [29]:
df=pd.DataFrame(fins, columns=['Label'])

In [38]:
imageid=range(1,28001)
imageid=list(imageid)

In [45]:
df['ImageId']=imageid
df.head()
f=open('submission.csv', 'w')
df.to_csv(f)